# Prepping for Modeling

Add final columns (document meta) and convert categorical items to text/dummy variables.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import random
import matplotlib
import matplotlib.cm as cm
import seaborn as sns
import datetime
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, \
    GradientBoostingClassifier, ExtraTreesClassifier, BaggingRegressor, \
    ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import confusion_matrix
import matplotlib.patheffects as path_effects
from matplotlib import cm as cm2
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score \
#     confusion_matrix, roc_curve, roc_auc_score, precision_score, recall_score
from sklearn.cross_validation import train_test_split, KFold, cross_val_score



matplotlib.style.use('ggplot') 
%matplotlib inline

/Users/alex/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
d_meta = pd.read_csv('./data/documents_meta.csv')


In [3]:
d_meta.isnull().sum()

document_id           0
source_id          2518
publisher_id      64024
publish_time    1011118
dtype: int64

In [4]:
len(d_meta)

2999334

Will not use publish_time, too much data missing. Will remove nan for other two columns.

In [ ]:
d_meta.dropna(axis = 0,inplace = True,subset=['publisher_id'])

In [ ]:
len(d_meta)

In [ ]:
d_meta.isnull().sum()

once data is loaded, use labelencoder to classify what is needed


how to determine favorite choice in a set???



#great stuff here: 
http://localhost:8888/notebooks/TA_reviews/Week_12/tree-based_methods/sklearn_tree-based_methods.ipynb

In [ ]:
events_train = pd.read_csv('./data/events_train.csv')
events_test = pd.read_csv('./data/events_test.csv')


In [ ]:
events_train.info()

In [ ]:
events_train.head(2)

In [ ]:
events_train.date_local[3]

In [ ]:
def time_bucket(x):
    hour = int(str(x))[11:13]
    if hour >6 and hour <12:
        s = 'morning'
    elif hour > 11 and hour <18:
        s = 'afternoon'
    elif hour > 17 and hour < 23:
        s = 'evening'
    else:
        s = 'overnight'
    return s

def date_bucket(x):
    return int(str(x))[9:11] # this is the date space in datetime


In [ ]:
# Set the period of time (morning,afternoon, etc...)
events_train['period'] = [time_bucket(rows['date_local']) for index,\
                        rows in events_train.iterrows()]
events_test['period'] = [time_bucket(rows['date_local']) for index,\
                       rows in events_test.iterrows()]

In [ ]:
# Set the day of month (all occur in June 2016)
events_train['d_month'] = [date_bucket(rows['date_local']) for index,\
                        rows in events_train.iterrows()]
events_test['d_month'] = [date_bucket(rows['date_local']) for index,\
                       rows in events_test.iterrows()]

In [ ]:
events_train = events_train.merge(d_meta,on='document_id',how='left', suffixes=('_l','_r'))
events_test = events_test.merge(d_meta,on='document_id',how='left', suffixes=('_l','_r'))

In [ ]:
events_test.columns

In [ ]:
rename = ['display_id', 'ad_id', 'clicked', 'uuid', 'docment_id',
       'platform', 'geo_location', 'contry', 'state', 'DMA',
       'date_local', 'ad_docment_id', 'campaign_id', 'advertiser_id',
       'Dt', 'Dc', 'De', 'D_tot', 'period', 'd_month', 'sorce_id_doc', 'pblisher_id_doc',
       'pblish_time_doc']

In [ ]:
events_train.columns = rename
events_test.columns = rename

In [ ]:
events_test.head(2)

In [ ]:
events_train = pd.merge(events_train, d_meta, how='left',\
                       left_on=['ad_docment_id'], right_on=['document_id'])
events_test = pd.merge(events_test, d_meta, how='left',\
                       left_on=['ad_docment_id'], right_on=['document_id'])


In [ ]:
# save this initial version of consolidated table

events_train.to_csv('./data/events_train1.csv',index=False)
events_test.to_csv('./data/events_test1.csv',index=False)

In [5]:
# to reload csv files when redoing this notebook:

events_train = pd.read_csv('./data/events_train1.csv')
events_test = pd.read_csv('./data/events_test1.csv')


In [6]:
for i in events_train.columns:
    print('{0} : {1}'.format(i,events_train[i].nunique()))

display_id : 50000
ad_id : 36529
clicked : 2
uuid : 49972
docment_id : 26728
platform : 3
geo_location : 337
contry : 1
state : 51
DMA : 210
date_local : 50000
ad_docment_id : 17842
campaign_id : 12660
advertiser_id : 2183
Dt : 26521
Dc : 19995
De : 1751
D_tot : 45651
period : 1
d_month : 10
sorce_id_doc : 1917
pblisher_id_doc : 426
pblish_time_doc : 3735
document_id : 8313
source_id : 902
publisher_id : 458
publish_time : 2539


ad's don't seem to have source/publisher info

**solution set (Y)** <br>
* clicked

** Columns for X: **<br>
* platform
* state
* DMA
* campaign_id
* advertiser_id
* D_tot: docuemnt score/vector on topic/subject correlation ***[only non-classifier]***
* period: morning, afternoon, etc...
* sorce_id_doc: site sub page, eg. edition.cnn.com
* pblisher_id_doc: site host, eg. cnn.com


In [7]:
# Break out X_train and X_test from main events dataframe

X_train = events_train[['platform', 'state', 'DMA', 'campaign_id', 'advertiser_id',\
                        'D_tot', 'period', 'sorce_id_doc', 'pblisher_id_doc']]
X_test = events_test[['platform', 'state', 'DMA', 'campaign_id', 'advertiser_id',\
                        'D_tot', 'period', 'sorce_id_doc', 'pblisher_id_doc']]

Y_train = events_train[['clicked']]
Y_test = events_test[['clicked']]

In [8]:
X_train.tail(6)

,platform,state,DMA,campaign_id,advertiser_id,D_tot,period,sorce_id_doc,pblisher_id_doc
257018,2,NJ,501,21573,210,0.068102,overnight,483.0,1199.0
257019,2,NJ,501,26260,2151,0.109092,overnight,483.0,1199.0
257020,2,TX,635,9765,372,0.000000,overnight,7270.0,58.0
257021,2,TX,635,17012,1627,1.804419,overnight,7270.0,58.0
257022,2,TX,635,22125,1685,1.312135,overnight,7270.0,58.0
257023,2,TX,635,19233,1992,0.000000,overnight,7270.0,58.0


In [9]:
# with open('X_train.pkl', 'w') as picklefile:
#    pickle.dump(X_train, picklefile)

In [10]:
# Shorten column names prior to making dummy variables

X_train.columns

Index([u'platform', u'state', u'DMA', u'campaign_id', u'advertiser_id',
       u'D_tot', u'period', u'sorce_id_doc', u'pblisher_id_doc'],
      dtype='object')

In [11]:
new_names = ['pltfrm', 'state', 'DMA', 'camp_id', 'advert_id', 'D_tot',\
             'prd', 'src_id', 'pbl_id']

In [12]:
X_train.columns = new_names
X_test.columns = new_names

In [13]:
X_train.head()

,pltfrm,state,DMA,camp_id,advert_id,D_tot,prd,src_id,pbl_id
0,2,CA,803,10321,555,0.000000,overnight,6509.0,206.0
1,2,CA,803,14312,1890,0.000000,overnight,6509.0,206.0
2,2,CA,803,19194,1458,0.000000,overnight,6509.0,206.0
3,2,CA,803,17146,1929,0.000000,overnight,6509.0,206.0
4,2,PA,504,12404,2151,1.758876,overnight,1032.0,1025.0


In [14]:
# change numbers to strings before get_dummies

num_str = ['pltfrm', 'DMA', 'camp_id', 'advert_id', 'src_id', 'pbl_id']

for s in num_str:
    print(s)
    X_train[s] = X_train[s].apply(str)
    X_test[s] = X_test[s].apply(str)

pltfrm
DMA
camp_id


/Users/alex/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/alex/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


advert_id
src_id
pbl_id


In [15]:
X_train.state[5]

'PA'

In [16]:
X_train = pd.get_dummies(X_train, prefix=['pltfrm', 'state', 'DMA', 'camp_id', 'advert_id',\
                                          'prd','src_id', 'pbl_id'])
X_test = pd.get_dummies(X_test, prefix=['pltfrm', 'state', 'DMA', 'camp_id', 'advert_id',\
                                        'prd','src_id', 'pbl_id'])


In [ ]:
X_train.to_csv('./data/X_train.csv',index=False)
X_test.to_csv('./data/X_test.csv',index=False)

In [ ]:
X_train.head()